In [29]:
import xgboost as xgb
from xgboost import XGBClassifier
import catboost as ctb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import time
import joblib
from sklearn.ensemble import VotingClassifier

In [21]:
train_col_drop = ['id', 'CustomerId', 'Surname', 'Exited']
X_train = data_train.drop(columns = train_col_drop)
X_train_dum = pd.get_dummies(X_train)
y_train = data_train['Exited']

test_col_drop = ['id', 'CustomerId', 'Surname']
X_test = data_test.drop(columns = test_col_drop)
X_test_dum = pd.get_dummies(X_test)

In [ ]:
parametrs = {
    "learning_rate" : [0.01, 0.1, 0.2, 0.3],
    "n_estimators" : [30, 50, 70, 100, 120, 200],
    "max_depth" : [5, 7, 9, 11, 15, 20, 25, 50]
}

xgb_model = XGBClassifier()
grid_search = GridSearchCV(estimator=xgb_model, param_grid=parametrs, cv=10, scoring='roc_auc', verbose=1, n_jobs=-1)

grid_search.fit(X_train_dum, y_train)

Fitting 10 folds for each of 192 candidates, totalling 1920 fits


In [11]:
print(grid_search.best_params_)

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 70}


In [50]:
# best_of_grid_xgb_model = grid_search.best_estimator_
model2.fit(X_train_dum, y_train)
best_of_grid_xgb_model = model2

In [51]:
best_xgb_predict = best_of_grid_xgb_model.predict_proba(X_test_dum)
subm_best_of_grid_xgb = {
            'id' : data_test['id'],
            'Exited' : best_xgb_predict[:, 1]}

submit_9 = pd.DataFrame(data = subm_best_of_grid_xgb)
submit_9.to_csv('submission_solo_xgb_l2_5.csv', index = False)

Попробуем оценить catboost и xgboost на кросс-валидацииfrom sklearn.model_selection import train_test_split

In [38]:
#Данные для валидации

X_train_subset_new, X_val, target_train_subset_new, target_val = train_test_split(X_train_dum, y_train, test_size = 0.2, random_state = 42)

In [58]:
data_train.head(2)

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15653521.0,Nkemakonam,667.0,Germany,Male,33.0,3.0,131769.04,1.0,1.0,1.0,162719.69,0.0
1,1,15699005.0,Chiekwugo,614.0,France,Female,31.0,2.0,110615.47,1.0,1.0,1.0,181879.56,0.0


In [22]:
X_train_catboost = data_train.drop(columns = ['id', 'CustomerId', 'Surname', 'Exited'])
y_train_catboost = data_train['Exited']

X_test_carboost = data_test.drop(columns = test_col_drop)
categorical_columns = ['Geography', 'Gender']

In [ ]:
# Определяем модель
catboost_model = CatBoostClassifier(verbose=100, cat_features = categorical_columns, task_type='GPU')  # verbose=0 отключает вывод информации

# Определяем сетку гиперпараметров для перебора
param_grid_catboost = {
    'learning_rate': [0.05, 0.2, 0.6],
    'depth': [4, 6, 8],             # Глубина дерева в CatBoost
    'n_estimators': [200, 1000], # Количество итераций (аналог n_estimators)
    'l2_leaf_reg' : [2, 5, 10] 
}

# Инициализируем GridSearchCV с использованием кросс-валидации (cv)
grid_search_catboost = GridSearchCV(estimator = catboost_model, param_grid = param_grid_catboost, 
                           cv=15, scoring='roc_auc', verbose=1)

# Обучаем модель с перебором гиперпараметров
grid_search_catboost.fit(X_train_catboost, y_train_catboost)

Fitting 15 folds for each of 54 candidates, totalling 810 fits
0:	learn: 0.6438055	total: 140ms	remaining: 27.8s
100:	learn: 0.2539091	total: 1.37s	remaining: 1.34s
199:	learn: 0.2456726	total: 2.45s	remaining: 0us
0:	learn: 0.6429150	total: 11.3ms	remaining: 2.25s
100:	learn: 0.2531079	total: 1.02s	remaining: 1s
199:	learn: 0.2443086	total: 2.04s	remaining: 0us
0:	learn: 0.6420031	total: 11ms	remaining: 2.18s
100:	learn: 0.2530687	total: 1.05s	remaining: 1.03s
199:	learn: 0.2442709	total: 2.06s	remaining: 0us
0:	learn: 0.6420956	total: 11.8ms	remaining: 2.35s
100:	learn: 0.2526163	total: 1.06s	remaining: 1.04s
199:	learn: 0.2439991	total: 2.08s	remaining: 0us
0:	learn: 0.6423241	total: 10.9ms	remaining: 2.18s
100:	learn: 0.2523566	total: 1.07s	remaining: 1.05s
199:	learn: 0.2437839	total: 2.08s	remaining: 0us
0:	learn: 0.6422679	total: 10.7ms	remaining: 2.13s
100:	learn: 0.2521342	total: 1s	remaining: 986ms
199:	learn: 0.2432297	total: 1.97s	remaining: 0us
0:	learn: 0.6417786	total: 1

In [28]:
# Находим лучшие параметры
print("Лучшие параметры: ", grid_search_catboost.best_params_)
print("Лучшая точность: ", grid_search_catboost.best_estimator_)

Лучшие параметры:  {'depth': 4, 'l2_leaf_reg': 5, 'learning_rate': 0.05, 'n_estimators': 200}
Лучшая точность:  <catboost.core.CatBoostClassifier object at 0x78c54d7230a0>


In [27]:
best_catboost_from_grid_search = grid_search_catboost.best_estimator_


catboost_pred = best_catboost_from_grid_search.predict_proba(X_test_carboost)
catboost_pred
subm_best_of_grid_catboost = {
            'id' : data_test['id'],
            'Exited' : catboost_pred[:, 1]}

submit_10_catboost = pd.DataFrame(data = subm_best_of_grid_catboost)
submit_10_catboost.to_csv('submission_21_catboost.csv', index = False)

In [30]:
joblib.dump(grid_search_catboost, 'grid_search_catboost.pkl')

['grid_search_catboost.pkl']

In [6]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [150]:
model1 = CatBoostClassifier(learning_rate=0.1, depth=6, iterations=100, random_state=42, verbose=0, cat_features=categorical_columns, task_type='GPU')
model2 = CatBoostClassifier(learning_rate=0.2, depth=6, iterations=100, random_state=42, verbose=0, cat_features=categorical_columns, task_type='GPU')
model3 = CatBoostClassifier(learning_rate=0.1, depth=3, iterations=100, random_state=42, verbose=0, cat_features=categorical_columns, task_type='GPU')
model4 = CatBoostClassifier(learning_rate=0.3, depth=5, iterations=100, random_state=42, verbose=0, cat_features=categorical_columns, task_type='GPU')
model5 = CatBoostClassifier(learning_rate=0.4, depth=4, iterations=100, random_state=42, verbose=0, cat_features=categorical_columns, task_type='GPU')


model6 = CatBoostClassifier(learning_rate=0.1, depth=6, iterations= 100, random_state=42, verbose=0, cat_features=categorical_columns, task_type='GPU')  # Увеличиваем итерации и глубину
model7 = CatBoostClassifier(learning_rate=0.1, depth=4, iterations=100, random_state=42, verbose=0, cat_features=categorical_columns, task_type='GPU')  # Меньше итераций, больше глубины
model8 = CatBoostClassifier(learning_rate=0.15, depth=7, iterations= 100, random_state=42, verbose=0,   cat_features=categorical_columns, task_type='GPU')  # Больше регуляризации
model9 = CatBoostClassifier(learning_rate=0.25, depth=3, iterations=100, random_state=42, verbose=0, cat_features=categorical_columns, task_type='GPU')  # Глубже дерево, больше итераций
model10 = CatBoostClassifier(learning_rate=0.35, depth=5, iterations=100, random_state=42, verbose=0, cat_features=categorical_columns, task_type='GPU')  # Меньше итераций, средняя глубина

# Создаем ансамбль

ensemble = VotingClassifier(
    estimators=[('cat1', model1), ('cat2', model2), ('cat3', model3), ('cat4', model4), ('cat5', model5),
               ('cat6', model6), ('cat7', model7), ('cat8', model8), ('cat9', model9), ('cat10', model10)],
    voting='soft',  # 'soft' для усреднения вероятностей, 'hard' для большинства голосов
)

ensemble.fit(X_train_catboost_modified, y_train_catboost)

VotingClassifier(estimators=[('cat1',
                              <catboost.core.CatBoostClassifier object at 0x7bf4084b6ef0>),
                             ('cat2',
                              <catboost.core.CatBoostClassifier object at 0x7bf4084b72b0>),
                             ('cat3',
                              <catboost.core.CatBoostClassifier object at 0x7bf4084b70a0>),
                             ('cat4',
                              <catboost.core.CatBoostClassifier object at 0x7bf4084b5db0>),
                             ('cat5',
                              <catboost.core.CatBoostClassifier object at 0x7bf408...0>),
                             ('cat6',
                              <catboost.core.CatBoostClassifier object at 0x7bf4084b6230>),
                             ('cat7',
                              <catboost.core.CatBoostClassifier object at 0x7bf4084b6470>),
                             ('cat8',
                              <catboost.core.CatBoostClassifier object at 0x7bf4084b5f90>),
                             ('cat9',
                              <catboost.core.CatBoostClassifier object at 0x7bf4084b57b0>),
                             ('cat10',
                              <catboost.core.CatBoostClassifier object at 0x7bf4084b5780>)],
                 voting='soft')

In [151]:
y_pred_ensemble = ensemble.predict_proba(X_test_carboost_modified)

In [152]:
y_pred_ensemble

array([[0.95231764, 0.04768236],
       [0.79126622, 0.20873378],
       [0.98050579, 0.01949421],
       ...,
       [0.8928726 , 0.1071274 ],
       [0.98928435, 0.01071565],
       [0.24573837, 0.75426163]])

In [155]:
subm_best_of_grid_catboost = {
            'id' : data_test['id'],
            'Exited' : y_pred_ensemble[:, 1]}

submit_catboost_mod = pd.DataFrame(data = subm_best_of_grid_catboost)
submit_catboost_mod.to_csv('submit_catboost_mod_12.csv', index = False)

In [156]:
describe_pred = pd.read_csv('/kaggle/working/submit_catboost_mod_12.csv')
describe_pred.describe()

,id,Exited
count,10000.00000,10000.000000
mean,19999.50000,0.200414
std,2886.89568,0.281662
min,15000.00000,0.004701
25%,17499.75000,0.018005
50%,19999.50000,0.058437
75%,22499.25000,0.246133
max,24999.00000,0.991657


In [143]:
describe_pred = pd.read_csv('/kaggle/working/submit_catboost_mod_10.csv')
describe_pred.describe()

,id,Exited
count,10000.00000,10000.000000
mean,19999.50000,0.198770
std,2886.89568,0.294269
min,15000.00000,0.000941
25%,17499.75000,0.012391
50%,19999.50000,0.044212
75%,22499.25000,0.248828
max,24999.00000,0.998736


In [21]:
y_pred_ensemble = ensemble.predict_proba(X_test_carboost)

In [56]:
y_pred_ensemble

array([[0.88333786, 0.11666214],
       [0.96159434, 0.03840566],
       [0.91023415, 0.08976585],
       ...,
       [0.93795028, 0.06204972],
       [0.98813115, 0.01186885],
       [0.98018656, 0.01981344]])

In [103]:
importances = []
for clf in ensemble.estimators_:
    if hasattr(clf, 'feature_importances_'):
        importances.append(clf.feature_importances_)

In [104]:
importances = np.mean(importances, axis = 0)
importances

array([ 5.28754475,  5.95934411,  2.75705598, 24.7890299 ,  1.59664375,
       10.42451592, 35.58400915,  7.67762532,  5.92423113])

In [144]:
X_train_catboost_modified = X_train.drop(columns = 'Balance')
categorical_columns = ['Geography', 'Gender']

X_test_carboost_modified = data_test.drop(columns = test_col_drop + ['Balance'])
X_train_catboost_modified.head(5)

,CreditScore,Geography,Gender,Age,Tenure,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,667.0,Germany,Male,33.0,3.0,1.0,1.0,1.0,162719.69
1,614.0,France,Female,31.0,2.0,1.0,1.0,1.0,181879.56
2,683.0,Germany,Female,24.0,6.0,2.0,1.0,0.0,109688.82
3,678.0,France,Female,38.0,9.0,1.0,1.0,0.0,122823.84
4,588.0,Spain,Female,39.0,3.0,2.0,1.0,1.0,136910.18


In [73]:
X_test_carboost_modified.head(5)

,CreditScore,Geography,Gender,Age,Balance,NumOfProducts,IsActiveMember,EstimatedSalary
0,821.0,Spain,Male,32.0,0.00,1.0,1.0,120893.85
1,634.0,France,Male,28.0,129299.28,1.0,0.0,179655.85
2,713.0,France,Male,42.0,0.00,2.0,1.0,80552.12
3,611.0,France,Female,38.0,0.00,1.0,1.0,151335.24
4,724.0,France,Male,29.0,0.00,1.0,1.0,88724.49


In [56]:
ensemble = pd.read_csv('/kaggle/input/ensemble/ensemble_catboost_3_jupyter.csv')
catboost_weight = pd.read_csv('/kaggle/working/submission_24_weight1_catboost.csv')
ensemble_with_weight = (ensemble + catboost_weight) / 2

ensemble_with_weight.to_csv('ensemble_with_weight.csv', index = False)